In [26]:
from collections import Counter
import dspy
from dspy import OpenAI, settings
from dspy.teleprompt import MIPROv2, BootstrapFewShot
from dspy.evaluate.evaluate import Evaluate
from dspy import ColBERTv2
from dsp.utils import print_message, normalize_text
from dotenv import load_dotenv
import os
load_dotenv("/media/uberdev/ddrv/gitFolders/python_de_learners_data/.env")

True

In [2]:
def f1_score_01(prediction, ground_truth):
    prediction_tokens = [normalize_text(elem) for elem in prediction.split("|")]
    ground_truth_tokens = [normalize_text(elem) for elem in ground_truth.split("|")]
    
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    
    if len(prediction_tokens) == len(ground_truth_tokens) == 0:
        print_message("\n#> F1 Metric: Rare edge case of len(prediction_tokens) == len(ground_truth_tokens) == 0.\n")
    
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1

Key Idea is to ping the LLM multiple times for the same prediction
and then get the f1_score of the prediction.

Keep track of the calls to LLMs inside phoenix using the instrumentation

    - Need to create a seperate project inside the Phoenix server for this exploration

    - Log the traces to that Project

    - Use the logs to further understand Metric functions

    - Move farther into OpenTelemetry

In [6]:
from openinference.instrumentation.dspy import DSPyInstrumentor
# instruments the internal calls in DSPy library
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
# help to get the span of http requests to the APIs
from opentelemetry.sdk import trace as trace_sdk
#processes the data collected from the spans
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.resource import ResourceAttributes
import phoenix as px

INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix
INFO:phoenix.inferences.inferences:Dataset: phoenix_inferences_a734833c-622d-4ca2-8379-115ad2d89b80 initialized


In [7]:
endpoint = "http://127.0.0.1:6006/v1/traces"
# resource = Resource(attributes={})

In [8]:
client = px.Client(endpoint=endpoint)

INFO:httpx:HTTP Request: GET http://127.0.0.1:6006/v1/traces/arize_phoenix_version "HTTP/1.1 200 OK"


In [9]:
# https://docs.arize.com/phoenix/tracing/how-to-tracing/trace-a-deployed-app
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'dspy-f1-score'
})

In [10]:
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)

In [11]:
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)

DSPyInstrumentor().instrument(skip_dep_check=True) # here where DSPy is instrumented

In [21]:
# Need to check if the dspy is writing logs
import os
llm = OpenAI(model='gpt-4o-mini',
             api_key=os.environ['OPENAI_API_KEY'],
             max_tokens=2000)

In [22]:
colbertv2_wiki17_abstracts = ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
settings.configure(lm=llm, rm=colbertv2_wiki17_abstracts)

In [23]:
llm("Let's see if you are logging to the console")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


["I don't have the capability to log to the console or perform any actions outside of providing text-based responses. However, I can help you with code examples or explain how to log to the console in various programming languages. Let me know what you need!"]

In [12]:
def F1(prediction, answers_list):
    # list of answers are entering for a single prediction
    assert isinstance(answers_list, list)
    return max(f1_score_01(prediction, ans) for ans in answers_list)

def answer_match(prediction, answers, frac=1.0):
    return F1(prediction, answers) >= frac

def answer_f1_match_01(example, pred, trace=[], frac=0.95):
    assert isinstance(example.answer, (str, list))
    print(f"Looking at the traces, {trace}")
    if isinstance(example.answer, str):
        return answer_match(pred.answer, [example.answer], frac=frac)
    else:
        return answer_match(pred.answer, example.answer, frac=frac)


In [13]:
class NewsCategorization(dspy.Signature):
    news_body = dspy.InputField(desc="The body of the news to be categorized")
    answer = dspy.OutputField(desc="Should be 'fake' or 'real'")

class CoTCombined(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(NewsCategorization)
        self.history = []  # This will store the history of operations

    def forward(self, news_body):
        pred_list = []
        for news in news_body.split("|"):
            pred_one = self.prog(news_body=news)
            pred_list.append(pred_one.answer)
            self.history.append(f"Processed news: {news}, Prediction: {pred_one.answer}")
        return dspy.Prediction(answer="|".join(pred_list))

    def inspect_history(self, n=None):
        """
        Return the last n entries of the history. If n is None, return the entire history.
        """
        if n is None:
            return self.history
        else:
            return self.history[-n:]

In [14]:
class CustomExample:
    def __init__(self, news_body, answer):
        self.news_body = news_body
        self.answer = answer
        self._dict = {"news_body": news_body,
                      "answer": answer}

    def with_inputs(self, input_key):
        return self

    def inputs(self):
        return {"news_body": self.news_body}

    def items(self):
        return self._dict.items()

    def copy(self):
        return CustomExample(self.news_body, self.answer)

    def get(self, key, default=None):
        return self._dict.get(key, default)

    def __iter__(self):
        return iter(self._dict)

    def __contains__(self, key):
        return key in self._dict

    def __getitem__(self, key):
        return self._dict[key]

    def keys(self):
        return self._dict.keys()

    def values(self):
        return self._dict.values()


In [15]:
custom_trainset = [
    CustomExample("Fake news body 1", "fake"),
    CustomExample("Real news body 2", "real"),
    CustomExample("Fake news body 3", "fake")
]

custom_devset = [
    CustomExample("Real news body 4", "real"),
    CustomExample("Fake news body 5", "fake")
]

In [18]:
print("Starting to build the optimizer")
model_to_generate_prompts = llm
model_that_solves_task = CoTCombined()
your_defined_metric = answer_f1_match_01
num_new_prompts_generated = 10
prompt_generation_temperature = 0.7

Starting to build the optimizer


In [19]:
teleprompter = MIPROv2(
    prompt_model=model_to_generate_prompts,
    task_model=model_that_solves_task,
    metric=your_defined_metric,
    num_candidates=num_new_prompts_generated,
    init_temperature=prompt_generation_temperature
)
print("Teleprompter assembled")

Teleprompter assembled


In [20]:
kwargs = dict(num_threads=1,
              display_progress=True,
              display_table=0)

In [24]:
compiled_program_optimized_bayesian_signature = teleprompter.compile(
    CoTCombined(), 
    trainset=custom_trainset,
    valset=custom_devset,
    num_batches=30,
    max_bootstrapped_demos=3, 
    max_labeled_demos=5, 
    eval_kwargs=kwargs,
    minibatch=True,
    program_aware_proposer=True
)


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 3 examples in train set * 3 full evals = 759 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may co

Do you wish to continue? (y/n):  y


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


summary: Prediction(
    summary="The dataset prominently explores technological advancements, particularly in AI and machine learning, and their societal impacts, specifically in healthcare, business, and education. It emphasizes detailed, persuasive content that balances depth with conciseness, aimed at engaging and educating a diverse audience about technology's role while focusing on user experiences. This suggests its use in educational platforms or discussions aimed at influencing opinions on technology adoption."
)
DATA SUMMARY: The dataset prominently explores technological advancements, particularly in AI and machine learning, and their societal impacts, specifically in healthcare, business, and education. It emphasizes detailed, persuasive content that balances depth with conciseness, aimed at engaging and educating a diverse audience about technology's role while focusing on user experiences. This suggests its use in educational platforms or discussions aimed at influencing 

 33%|███▎      | 1/3 [00:01<00:02,  1.44s/it]

...

INFO:dspy.evaluate.evaluate:2024-08-17T11:35:39.192288Z [info     ] Average Metric: 2 / 3 (66.7%)  [dspy.evaluate.evaluate] filename=evaluate.py lineno=200



FULL TRACE
Average Metric: 1 / 1  (100.0): 100%|██████████| 1/1 [00:00<00:00,  5.43it/s]

INFO:dspy.evaluate.evaluate:2024-08-17T11:35:39.404782Z [info     ] Average Metric: 1 / 1 (100.0%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=200


[I 2024-08-17 17:05:39,429] Trial 29 finished with value: 66.67 and parameters: {'0_predictor_instruction': 6, '0_predictor_demos': 5}. Best is trial 0 with value: 100.0.


...
Score 66.67


In [25]:
compiled_program_optimized_bayesian_signature.save("extracting_compiled_program.json")

[('prog', Predict(StringSignature(news_body -> rationale, answer
    instructions='Given the fields `news_body`, produce the fields `answer`.'
    news_body = Field(annotation=str required=True json_schema_extra={'desc': 'The body of the news to be categorized', '__dspy_field_type': 'input', 'prefix': 'News Body:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': "Should be 'fake' or 'real'", '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)))]
